# Market Conditions Data Processing (市況データ処理)

## Overview
This notebook processes market conditions data from multiple countries, handling various date formats and currency notations. It converts daily/weekly price data into a standardized monthly format.

### Input Data
- Source: Excel file with multiple sheets
- Countries: Brazil, Thailand, USA, China
- Date Range: Varies by country
- Currencies: Different units by country

### Output Format
- Monthly averaged data
- Standardized currency columns
- Clean, consistent index in yyyymm format

## 1. Setup and Data Loading

In [1]:
# Import required libraries
from google.colab import drive
import pandas as pd
from datetime import datetime
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define file paths and sheet names
FILE_PATH = "/content/drive/Shareddrives/125-2日本ハム-業務委託共有/前処理作業ファイル for Joseph-san/Before Preprocessing/2024-06断面データ/★O PREÇO （24.5.29）.xlsx"
BRASIL_SHEET = "ブラジル週間価格（2013年5月-）"
THAI_SHEET = "タイ週間価格（2014年1月-）"
USA_SHEET = "アメリカ週間価格（2014年12月-）"
CHINA_SHEET = "中国鴨週間価格（2023年5月-）"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Define Helper Functions

In [2]:
# Define Helper Functions
def get_first_and_last_year(sheet_name):
    """Extract first and last years from a sheet."""
    data = pd.read_excel(FILE_PATH, sheet_name=sheet_name, header=None)
    # Fix deprecation warning by using pd.Series.str.contains directly
    year_mask = data.iloc[:, 0].str.contains(r'\d{4}年', na=False)

    first_year_position = year_mask.iloc[::1].idxmax()
    first_year = int(data.iloc[first_year_position, 0][:4])

    last_year_position = year_mask.iloc[::-1].idxmax()
    last_year = int(data.iloc[last_year_position, 0][:4])

    return first_year, last_year

def get_table_position(sheet_name, year):
    """Get start and end positions for a year's data."""
    data = pd.read_excel(FILE_PATH, sheet_name=sheet_name, header=None)

    start_index = data.index[data.iloc[:, 0] == f"{year}年"].tolist()[0] + 1
    try:
        end_index = data.index[data.iloc[:, 0] == f"{year+1}年"].tolist()[0] - 1
    except IndexError:
        end_index = data.index.stop

    return {year: (start_index, end_index)}

def split_table_by_year(table_data, year, country):
    """Split table data if it contains multiple years."""
    # Fix deprecation warning by using pd.Series.str.contains directly
    contains_year = table_data.iloc[0].str.contains(r'\d{4}年', na=False)[1:]
    year_positions = contains_year[contains_year].index.tolist()

    if len(year_positions) > 1:
        print(f"Splitting {country}_{year} table into more tables")
        tables = {}
        for i in range(len(year_positions) - 1):
            start_col, end_col = year_positions[i], year_positions[i+1] - 1
            sub_table = table_data.iloc[1:, start_col:end_col+1]
            sub_table = pd.concat([table_data.iloc[1:, 0], sub_table], axis=1)
            tables[year-(i+1)] = sub_table

        last_sub_table = table_data.iloc[1:, year_positions[-1]:]
        last_sub_table = pd.concat([table_data.iloc[1:, 0], last_sub_table], axis=1)
        tables[year] = last_sub_table
        return tables
    else:
        return {year: table_data.iloc[1:]}

def set_first_column_as_index(tables):
    """Set the first column as index for all tables."""
    for year in tables:
        tables[year].set_index(tables[year].columns[0], inplace=True)
    return tables

def set_first_row_as_column_names(tables):
    """Use the first row as column names for all tables."""
    for year in tables:
        tables[year].columns = tables[year].iloc[0]
        tables[year] = tables[year].drop(tables[year].index[0])
    return tables

def transpose_tables(tables):
    """Transpose all tables."""
    return {year: table.transpose() for year, table in tables.items()}

def add_country_prefix_to_columns(tables):
    """Add country prefix to all column names."""
    for year in tables:
        tables[year].columns = [f"{country}_{col}" for col in tables[year].columns]
    return tables

def concatenate_country_dataframes(country_dict):
    """Concatenate tables for each country."""
    result = {}
    for country, tables in country_dict.items():
        result[country + "_df"] = pd.concat(tables.values())
    return result

## 3. Data Processing Functions

In [3]:
def get_all_year_tables(sheet_name):
    """Process all years for a given sheet."""
    first_year, last_year = get_first_and_last_year(sheet_name)
    all_tables = {}

    for year in range(first_year, last_year + 1):
        start_index, end_index = get_table_position(sheet_name, year)[year]
        table_data = pd.read_excel(
            FILE_PATH,
            sheet_name=sheet_name,
            header=None,
            skiprows=start_index-1,
            nrows=end_index-start_index+1
        )
        split_tables = split_table_by_year(table_data, year, sheet_name.split('（')[0])
        all_tables.update(split_tables)

    return all_tables

def process_dataframe(tables):
    """Clean and standardize dataframe format."""
    processed_tables = {}

    for year, df in tables.items():
        # Convert dates to YYYYMMDD format
        def convert_to_date(value):
            if isinstance(value, str) and value.endswith('月'):
                month_num = int(value[:-1])
                return f"{year}{month_num:02d}01"
            elif isinstance(value, (pd.Timestamp, datetime)):
                if value.year != year:
                    value = value.replace(year=year)
                return value.strftime('%Y%m%d')
            return value

        # Process dates and clean data
        df.index = df.index.map(convert_to_date)
        mask = df.index.astype(str).str.contains('平均') | df.index.isna()
        df = df[~mask]
        df.index.name = None

        processed_tables[year] = df

    return processed_tables

## 4. Process Country Data

In [4]:
# Dictionary mapping sheet names to country codes
sheet_dict = {
    BRASIL_SHEET: "ブラジル",
    THAI_SHEET: "タイ",
    USA_SHEET: "アメリカ",
    CHINA_SHEET: "中国"
}

# Process all sheets
country_dict = {}
for sheet_name, country in sheet_dict.items():
    # Process each country's data
    tables = get_all_year_tables(sheet_name)
    tables = set_first_column_as_index(tables)
    tables = set_first_row_as_column_names(tables)
    tables = transpose_tables(tables)
    tables = process_dataframe(tables)

    # Add country prefix and store results
    globals()[country] = tables
    add_country_prefix_to_columns(tables)
    country_dict[country] = tables

# Concatenate dataframes for each country
concatenated_dataframes = concatenate_country_dataframes(country_dict)

# Assign concatenated dataframes to variables
for variable_name, dataframe in concatenated_dataframes.items():
    globals()[variable_name] = dataframe

Splitting アメリカ週間価格_2014 table into more tables


## 5. Clean and Standardize Data

In [5]:
# Set pandas option to handle downcasting behavior
pd.set_option('future.no_silent_downcasting', True)

# Clean Thai data (remove problematic rows)
タイ_df = タイ_df[タイ_df.index != '-']

# Combine all country data
市況 = pd.concat([ブラジル_df, タイ_df, アメリカ_df, 中国_df], axis=1)

# Clean numeric data
for column in 市況.columns:
    try:
        # Convert to string and clean separators
        市況[column] = 市況[column].astype(str).str.replace('..', '.')

        # Create a mask for values to replace with NaN
        nan_mask = (市況[column] == 'nan') | \
                  (市況[column] == 'NaT') | \
                  (市況[column] == 'ＮＡ') | \
                  (市況[column] == '-')

        # Replace values using mask
        市況.loc[nan_mask, column] = np.nan

        # Convert to numeric
        市況[column] = pd.to_numeric(市況[column], errors='coerce')
    except ValueError as e:
        print(f"Error in column '{column}': {str(e)}")
        raise e

# Sort by date
市況 = 市況.sort_index()

## 6. Handle Special Cases: Brazil Currency Columns

In [6]:
# Convert to monthly data with updated resample parameter
市況.index = pd.to_datetime(市況.index, format='%Y%m%d')
monthly_data = 市況.resample('ME').mean()
monthly_data.index = monthly_data.index.strftime('%Y%m')

# Unify BRR/BRL columns
def unify_columns(df, brr_col, brl_col, new_col_name):
    df[new_col_name] = df[brr_col].fillna(df[brl_col])
    return df.drop([brr_col, brl_col], axis=1)

# List of columns to unify
column_pairs = [
    ('ブラジル_生鶏価格(SP、BRR/KG )', 'ブラジル_生鶏価格(SP、BRL/KG )', 'ブラジル_生鶏価格_統合'),
    ('ブラジル_卸売価格(SP、BRR/KG )　月毎　翌月更新', 'ブラジル_卸売価格(SP、BRL/KG )　月毎　翌月更新', 'ブラジル_卸売価格_統合'),
    ('ブラジル_小売り価格(SP、BRR/KG )　月毎　翌月更新', 'ブラジル_小売り価格(SP、BRL/KG )　月毎　翌月更新', 'ブラジル_小売り価格_統合'),
    ('ブラジル_卵(SP BRR/ダース)', 'ブラジル_卵(SP BRL/ダース)', 'ブラジル_卵_統合'),
    ('ブラジル_為替　USD=BRR', 'ブラジル_為替　USD=BRL', 'ブラジル_為替_統合')
]

# Process each pair
monthly_data_cleaned = monthly_data.copy()
for brr_col, brl_col, new_col_name in column_pairs:
    monthly_data_cleaned = unify_columns(monthly_data_cleaned, brr_col, brl_col, new_col_name)

## 7. Export Results

In [7]:
# Define the CSV filename
csv_filename = '市況_mm.csv'

# Convert the DataFrame to a CSV file
monthly_data_cleaned.to_csv(csv_filename, index=True)

# Download the result
from google.colab import files
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>